<a href="https://colab.research.google.com/github/wangbxj1234/voxel/blob/main/collate_fn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch

In [2]:
def collate_fn(coord, feat, label):#(B,N,C)
    #coord, feat, label = list(zip(*batch))#(B,N,C)
    offset = [6,12]
    
    ####
    #coord, feat, label = torch.cat(coord,dim=0), torch.cat(feat,dim=0), torch.cat(label,dim=0) #但是属于不同batch的，就算坐标近也不能分进一个v。
    offset = torch.IntTensor(offset)##(N,3),(N,C),(N,),O

    ##to np
    coord, feat, label = coord.cpu().detach().numpy(),feat.cpu().detach().numpy(),label.cpu().detach().numpy()
    ##
    PN=0;
    voxel=[]
    for o in offset:
        c, f, l =coord[PN:int(o),:],feat[PN:int(o),:],label[PN:int(o),:]#(n,C),(n,C),(n,)
        discrete_coord = np.floor(c *4) 
        assert discrete_coord.ndim == 2
    # Floor first for negative coordinates
        arr = discrete_coord.copy()
        arr = arr.astype(np.uint64, copy=False)
        hashed_arr = np.uint64(14695981039346656037) * np.ones(arr.shape[0], dtype=np.uint64)
        for j in range(arr.shape[1]):
            hashed_arr *= np.uint64(1099511628211)
            hashed_arr = np.bitwise_xor(hashed_arr, arr[:, j])
        idx_sort = np.argsort(hashed_arr) 
        key_sort = hashed_arr[idx_sort] 
        _, count1 = np.unique(key_sort, return_counts=True)
        coord[PN:int(o),:],feat[PN:int(o),:],label[PN:int(o),:]= c[idx_sort],f[idx_sort],l[idx_sort]
        voxel.append(count1)
        PN=int(o)

    ##to torch
    coord, feat, label = torch.tensor(coord),torch.tensor(feat),torch.tensor(label)
    ##    

    return coord, feat, label, offset, voxel

In [3]:
#coord, feat, label=torch.rand(2,6,3),torch.rand(2,6,4),torch.rand(2,6,1)
coord, feat, label=torch.rand(2*6,3),torch.rand(2*6,4),torch.rand(2*6,1)
coord[0,:]=coord[1,:]
coord

tensor([[0.3205, 0.3627, 0.5357],
        [0.3205, 0.3627, 0.5357],
        [0.8643, 0.4473, 0.0057],
        [0.2726, 0.5545, 0.7501],
        [0.7521, 0.1422, 0.3147],
        [0.1669, 0.6799, 0.2549],
        [0.6682, 0.6269, 0.8652],
        [0.8147, 0.4467, 0.3710],
        [0.0742, 0.7375, 0.0370],
        [0.2847, 0.1096, 0.4617],
        [0.6015, 0.5169, 0.6452],
        [0.2654, 0.3807, 0.4452]])

In [4]:
coord, feat, label, offset, voxel = collate_fn(coord, feat, label)
print(coord.size(),feat.size(),label.size())

torch.Size([12, 3]) torch.Size([12, 4]) torch.Size([12, 1])


In [5]:
voxel

[array([1, 1, 1, 2, 1]), array([1, 1, 1, 1, 1, 1])]

In [8]:
voxel2=np.concatenate(voxel, axis=0)# 6-1=5
torch.tensor(voxel2)#好像实际操作只需要 voxel-wies的分界符，不需要 pcloud-wise的。

tensor([1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1])

In [12]:
#好像只需要 voxel-wise的分界符，不需要 pcloud-wise的。
voxel3=np.cumsum(voxel2)
torch.tensor(voxel3)

tensor([ 1,  2,  3,  5,  6,  7,  8,  9, 10, 11, 12])